In [27]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from collections import Counter

In [28]:
class Node:
    
    def __init__(self, X, y, feature = None, threshold=None, left=None, right=None, value=None):
        self.X = X
        self.y = y
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def check_leaf(self):
        return self.value is not None

In [29]:
class MyDecisionTreeClassifier:
    
    def __init__(self, min_samples_split = 2, max_depth = 2, feature_number = 0):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.path = None
        self.feature_number = feature_number
    
    def gini(self, groups, classes):
        '''
        A Gini score gives an idea of how good a split is by how mixed the
        classes are in the two groups created by the split.
        
        A perfect separation results in a Gini score of 0,
        whereas the worst case split that results in 50/50
        classes in each group result in a Gini score of 0.5
        (for a 2 class problem).
        '''
        
        gini_score = 1
        for i in groups:
            gini_score -= (i/classes)**2
        return gini_score
    
    def split_data(self, X, y) -> tuple[int, int]:
        _, num_features = X.shape
        general_gini = 1
        split_idx, split_thresh = None, None
        feat_idxs = np.random.choice(num_features, self.feature_number, replace=False)
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                left_child = np.argwhere(X_column <= threshold).flatten()
                right_child = np.argwhere(X_column > threshold).flatten()
                if len(left_child) == 0 or len(right_child) == 0:
                    continue
                left_gini = self.gini(np.bincount(y[left_child]), len(left_child))
                right_gini = self.gini(np.bincount(y[right_child]), len(right_child))
                current = (len(right_child)/len(y))*right_gini + (len(left_child)/len(y))*left_gini
                if current < general_gini:
                    general_gini = current
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh
    
    def build_tree(self, X, y, depth = 0):
        num_samples, _ = X.shape
        num_labels = len(np.unique(y))

        if (depth >= self.max_depth
            or num_labels == 1
            or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(X, y, value=leaf_value)

        best_feat, best_thresh = self.split_data(X, y)
        left_idxs = np.argwhere(X[:, best_feat] <= best_thresh).flatten()
        right_idxs = np.argwhere(X[:, best_feat] > best_thresh).flatten()
        left = self.build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self.build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(X, y, best_feat, best_thresh, left, right)

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def fit(self, X, y):
        if not self.feature_number:
            self.feature_number = X.shape[1]
        else:
            self.feature_number = min(X.shape[1], self.feature_number)
        self.path = self.build_tree(X, y)
    
    def predict(self, X_test):
        return np.array([self.traverse_tree(x, self.path) for x in X_test])
    
    def traverse_tree(self, x, node):
        if node.check_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

    def evaluate(self, X_test, y_test):
        return np.sum(X_test == y_test) / len(y_test)
    

In [34]:
iris = load_iris()
clf = MyDecisionTreeClassifier()
X, y = iris.data, iris.target
new_X, X_test, new_y, y_test = train_test_split(X, y, test_size=0.275)
clf.fit(new_X, new_y)
print(clf.evaluate(y_test, clf.predict(X_test)))

0.8809523809523809
